In [ ]:
!pip install -r requirements.txt

In [3]:
import json
from transformers import PreTrainedTokenizerFast, CamembertForTokenClassification, AutoModelForTokenClassification,TrainingArguments,DataCollatorForTokenClassification,Trainer
import yaml
import re
from datasets import load_dataset,Dataset, load_metric
import evaluate
import datasets
from sklearn.model_selection import train_test_split
import importlib
import ast
import pandas as pd
import torch
import accelerate

In [ ]:
with open('tokens.yml') as f:
    tokens = yaml.load(f, Loader=yaml.FullLoader)


def get_labels(tokens):
    labels = {}
    for token in tokens:
        labels[tokens[token]['start']] = token
    return labels

tag2literal = get_labels(tokens)

literal2tag = {tag2literal[key]:key for key in tag2literal}

In [6]:
df=pd.read_csv('data.csv')
dataset=load_dataset('csv',data_files='data.csv',num_proc=10)

Setting num_proc from 10 back to 1 for the train split to disable multiprocessing as it only contains one shard.


Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
features=datasets.Features(
                {
                    "text": datasets.Sequence(datasets.Value("string")),
                    "labels": datasets.Sequence(
                        datasets.features.ClassLabel(
                            num_classes=len(literal2tag),
                            names=literal2tag,
                        )
                    ),

                }
            )

In [8]:
print(features)

{'text': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'labels': Sequence(feature=ClassLabel(names=['age', 'birth_date', 'civil_status', 'education_level', 'employer', 'firstname', 'link', 'lob', 'maiden_name', 'nationality', 'observation', 'occupation', 'surname', 'surname_household'], id=None), length=-1, id=None)}

In [9]:
dataset = dataset['train'].map(lambda example: {"text": ast.literal_eval(example["text"]), "labels": ast.literal_eval(example["labels"])}, features=features)
dataset=dataset.train_test_split(test_size=0.2,seed=3)

Map:   0%|          | 0/23054 [00:00<?, ? examples/s]

In [11]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 18443
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 4611
    })
})

In [12]:
tokenizer=PreTrainedTokenizerFast.from_pretrained('camembert-base')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'CamembertTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [13]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

1

In [14]:
label_all_tokens = False
def tokenize_and_align_labels(examples):
    # Tokenize the text and add padding
    tokenized_data = tokenizer(examples["text"], padding=True, is_split_into_words=True, return_tensors='pt')
    # Initialize an empty list to store the labels
    processed_labels = []

    # Iterate over each label in the examples
    for i, label in enumerate(examples["labels"]):
        # Get the word IDs for the current example
        word_ids = tokenized_data.word_ids(batch_index=i)

        # Initialize variables to store the previous word index and the label IDs for the current example
        prev_word_idx = None
        label_ids = []

        # Iterate over each word ID in the current example
        for word_idx in word_ids:
            # If the word ID is None, it corresponds to a special token. Set the label ID to -100 to ignore it in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # If the word ID is different from the previous one, it corresponds to the first token of a word. Set the label ID to the current label.
            elif word_idx != prev_word_idx:
                label_ids.append(label[word_idx])
            # If the word ID is the same as the previous one, it corresponds to a subsequent token of the same word. Set the label ID to either the current label or -100, depending on the label\_all\_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)

            # Update the previous word index
            prev_word_idx = word_idx

        # Append the label IDs for the current example to the list of processed labels
        processed_labels.append(label_ids)

    # Add the processed labels to the tokenized data
    tokenized_data["labels"] = processed_labels

# Return the preprocessed data
return tokenized_data

In [15]:
tokenized_datasets=dataset.map(tokenize_and_align_labels, batched=True,num_proc=10)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=10):   0%|          | 0/18443 [00:00<?, ? examples/s]

Map (num_proc=10):   0%|          | 0/4611 [00:00<?, ? examples/s]

In [16]:
model=AutoModelForTokenClassification.from_pretrained('camembert-base', num_labels=14)
model.resize_token_embeddings(len(tokenizer)+1)

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of CamembertForTokenClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
print(tokenized_datasets)

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 18443
    })
    test: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4611
    })
})

In [21]:
args = TrainingArguments(
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
    output_dir='/home/onyxia/work'
)

In [22]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
metric1 = evaluate.load("precision")
metric2 = evaluate.load("recall")
metric3 = evaluate.load("f1")
#metric4 = evaluate.load("accuracy")

example=dataset['train'][4]
labels=example['labels']
test_prediction=[12, 5, 6, 1]

In [ ]:
metric1.compute(predictions=test_prediction, references=labels,average='micro')

In [24]:
metric = load_metric("seqeval")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=a60d3dd5fb963242c4429fc9fa4cb137ec07f8841c8ac2c88e2ca2b6acec806e
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


<ipython-input-24-26ba7ae7cb61>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [25]:
import numpy as np
import torch.nn.functional as F

label_list=literal2tag
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(F.softmax(torch.Tensor(predictions)), axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

"recall": metric2.compute(predictions=Value(true_predictions), references=Value(true_labels),average='micro'),
        "f1": metric3.compute(predictions=Value(true_predictions), references=Value(true_labels),average='micro'),
        "accuracy": metric4.compute(predictions=Value(true_predictions), references=Value(true_labels)),

In [26]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Training over 3 epoch

In [29]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.175700,0.124843,0.974406,0.973835,0.974121,0.974575
2,0.080600,0.081893,0.975229,0.975053,0.975141,0.975341
3,0.067600,0.077246,0.975002,0.974782,0.974892,0.975071


<ipython-input-25-d4a143bd0101>:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  predictions = np.argmax(F.softmax(torch.Tensor(predictions)), axis=2)
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: surname_household seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: firstname seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: occupation seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: link seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/l

TrainOutput(global_step=6918, training_loss=0.19944900568939763, metrics={'train_runtime': 678.977, 'train_samples_per_second': 81.489, 'train_steps_per_second': 10.189, 'total_flos': 804059068508004.0, 'train_loss': 0.19944900568939763, 'epoch': 3.0})

In [30]:
trainer.evaluate()

<ipython-input-25-d4a143bd0101>:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  predictions = np.argmax(F.softmax(torch.Tensor(predictions)), axis=2)


{'eval_loss': 0.07724618911743164,
 'eval_precision': 0.9750022561140691,
 'eval_recall': 0.9747823341002391,
 'eval_f1': 0.9748922827043245,
 'eval_accuracy': 0.9750710003155569,
 'eval_runtime': 9.4218,
 'eval_samples_per_second': 489.396,
 'eval_steps_per_second': 61.241,
 'epoch': 3.0}

In [28]:
tokenized_datasets['train']['input_ids'][0]

[5,
 84,
 5008,
 7593,
 470,
 27312,
 918,
 8,
 401,
 3294,
 6,
 32005,
 32005,
 32005,
 32005,
 32005,
 32005,
 32005,
 32005,
 32005,
 32005,
 32005,
 32005]

In [31]:
predictions=trainer.predict(tokenized_datasets['test'])

<ipython-input-25-d4a143bd0101>:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  predictions = np.argmax(F.softmax(torch.Tensor(predictions)), axis=2)


In [32]:
import torch
import torch.nn.functional as F
F.softmax(torch.Tensor(predictions.predictions[3]),dim=-1).argmax(dim=-1)

tensor([ 9, 13, 13,  5, 11,  6,  0,  9,  9,  9,  9,  9,  9,  4, 11,  9,  6,  6,
         9,  9,  9,  0,  0,  0,  0])

In [33]:
dataset['test']['labels'][:10]

[[13, 5, 11, 6, 0],
 [12, 5, 11, 6, 1, 9],
 [12, 5, 6, 0],
 [13, 5, 11, 0],
 [12, 5, 6, 0, 2],
 [13, 5, 11, 6, 1, 9],
 [12, 5, 11, 6, 1, 9],
 [12, 5, 11, 6, 1, 9],
 [12, 5, 11, 6, 4, 0],
 [12, 5, 11, 1, 2, 7]]

In [37]:
tokenized_datasets['test']['labels'][4]

[-100,
 12,
 -100,
 5,
 6,
 -100,
 0,
 2,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100]

In [38]:
tokenized_datasets['test']['text'][4]

['Faugère', 'Jean', 'leur fils', '36', 'Homme marié']

In [35]:
tokenizer.pad_token_id

32005